In [112]:
#dependencies
import requests
import pandas as pd
from config import fb_key
import numpy as np

ad_attributes = ['id', 'ad_snapshot_url', 'ad_creative_body', 'page_name', 'demographic_distribution', 'impressions', 'currency', 'spend']

In [118]:
# fb_ad_api class object for querying api
class fb_ad_api:
    def __init__(self, search='""'):
        #initialize object to contain lists for ad attributes
        self.id = []
        self.ad_snapshot_url = []
        self.ad_creative_body = []
        self.page_name = []
        self.demographic_distribution = []
        self.impressions = []
        self.currency = []
        self.spend = []
        #list comprehension from ad_attributes list to set returned attributes
        self.params = {
            'fields': ','.join(attribute for attribute in ad_attributes), 
            'ad_reached_countries': 'US', #countries where the ad is available
            'access_token': fb_key, #access token
            # 'ad_active_status': 'ALL',
            'limit': 100
            }
        self.params.update({'search_terms': search}) #add search term to params if one is provided
        self.base_url = 'https://graph.facebook.com/v6.0/ads_archive?' #set base API URL
    
    def call_api(self): #function to call api
        page_counter = 1
        page_limit = 3 # max num of pages to pull data from
        url = self.base_url

        while page_counter <= page_limit:

            #only include parameters for api call on page 1
            if page_counter == 1:
                response = requests.get(url, params = self.params).json()
            else:
                response = requests.get(url).json()
                
            results = response['data']

            
            for ad in results: #loop through each ad in api response page and append to attribute lists
                for attr in ad_attributes: #loop through each ad attribute and append to a list if present. Appends NaN if not present
                    if ad[f'{attr}']:
                        getattr(self, f'{attr}').append(ad[f'{attr}'])
                    else:
                        getattr(self, f'{attr}').append(np.nan)
                # try: 
                #     self.id.append(ad['id'])
                #     self.ad_snapshot_url.append(ad['ad_snapshot_url'])
                #     self.ad_creative_body.append(ad['ad_creative_body'])
                #     self.page_name.append(ad['page_name'])
                #     self.demographic_distribution.append(ad['demographic_distribution'])
                #     self.impressions.append(ad['impressions']['upper_bound'])
                #     self.currency.append(ad['currency'])
                #     self.spend.append(ad['spend']['upper_bound'])
                # except:
                #     break

            # break the loop if there isn't another page        
            if not(response['paging']['next']):
                break

            # set next api url according to paginated response    
            url = response['paging']['next']
            page_counter += 1
            print('Pulling page:' + str(page_counter))

    # make pandas df
    def make_df(self):
        results_df = pd.DataFrame({
            'Ad ID': self.id,
            'Ad URL': self.ad_snapshot_url,
            'Ad Text': self.ad_creative_body,
            'Hosted Page': self.page_name,
            'Impressions': self.impressions,
            'Currency': self.currency,
            'Ad Spending': self.spend
        })

        return results_df
        

In [119]:
search_results = fb_ad_api()
search_results.call_api()

Pulling page:2
Pulling page:3
Pulling page:4


In [123]:
search_results.make_df()

,Ad ID,Ad URL,Ad Text,Hosted Page,Impressions,Currency,Ad Spending
0,161547868597338,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,"{'lower_bound': '1000', 'upper_bound': '1999'}",USD,"{'lower_bound': '0', 'upper_bound': '99'}"
1,231634031267946,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,"{'lower_bound': '0', 'upper_bound': '999'}",USD,"{'lower_bound': '0', 'upper_bound': '99'}"
2,233098731244956,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,"{'lower_bound': '0', 'upper_bound': '999'}",USD,"{'lower_bound': '0', 'upper_bound': '99'}"
3,246740140011738,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,"{'lower_bound': '0', 'upper_bound': '999'}",USD,"{'lower_bound': '0', 'upper_bound': '99'}"
4,290585385271290,https://www.facebook.com/ads/archive/render_ad...,{{product.brand}},Vote Yes for Fair Tax,"{'lower_bound': '0', 'upper_bound': '999'}",USD,"{'lower_bound': '0', 'upper_bound': '99'}"
...,...,...,...,...,...,...,...
295,676305369814678,https://www.facebook.com/ads/archive/render_ad...,Healthcare and other essential workers are put...,SEIU,"{'lower_bound': '0', 'upper_bound': '999'}",USD,"{'lower_bound': '0', 'upper_bound': '99'}"
296,678090156345207,https://www.facebook.com/ads/archive/render_ad...,All workers — especially Black and brown essen...,SEIU,"{'lower_bound': '0', 'upper_bound': '999'}",USD,"{'lower_bound': '0', 'upper_bound': '99'}"
297,678855926282815,https://www.facebook.com/ads/archive/render_ad...,El presidente Trump ha fallado. No ha asegurad...,SEIU,"{'lower_bound': '0', 'upper_bound': '999'}",USD,"{'lower_bound': '0', 'upper_bound': '99'}"
298,695805207859555,https://www.facebook.com/ads/archive/render_ad...,Healthcare and other essential workers are put...,SEIU,"{'lower_bound': '0', 'upper_bound': '999'}",USD,"{'lower_bound': '0', 'upper_bound': '99'}"
